### Visión por Computadora I

Alumno:
   
    - Jorge Ceferino Valdez

Trabajo Práctico N° 3

- Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en Material_TPs/TP3/images a partir del template Material_TPs/TP3/template
1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.

Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección.


## Detección en todas las imagenes excepto la que tiene multiple detección.

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imutils

# Cargamos la plantilla, obtenemos sus bordes y la mostramos en pantalla
template = cv2.imread('../data/template/pattern.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template = cv2.Canny(template, 50, 200)  # Bordes de la plantilla
template_height, template_width = template.shape[:2]

# Cargamos las imagenes objetivos en una lista
images = ['../data/images/coca_logo_1.png',  \
          '../data/images/coca_logo_2.png',  \
          '../data/images/coca_retro_1.png', \
          '../data/images/coca_retro_2.png', \
          '../data/images/COCA-COLA-LOGO.jpg']

# Iteramos sobre las imagenes objetivos

for image in images:
    # Cargamos la imagen objetivo y la mostramos en pantalla
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # La variable found nos permitirá saber si encontramos o no la plantilla en la imagen
    found = None

    # Iteramos sobre 100 escalas diferentes de la imagen, entre 0.1 y 6.0
    for scale in np.linspace(0.1, 2.0, 10)[::-1]:
        # Redimensionamos la imagen según la escala actual
        resized = imutils.resize(gray, width=int(gray.shape[1] * scale))

        # Si la imagen redimensionada es más pequeña que la plantilla, entonces paramos de iterar
        if resized.shape[0] < template_height or resized.shape[1] < template_width:
            break

        # Calculamos el mapa de bordes de la imagen redimensionada, usamos Canny para ello.
        edged = cv2.Canny(resized, 50, 200)

        # Aplicamos el algoritmo de emparejamiento de plantillas de OpenCV para buscar la plantilla en la imagen redimensionada
        result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
        _, maxVal, _, maxLoc = cv2.minMaxLoc(result)

        # Calculamos el ratio entre la imagen la original y la imagen redimensionada
        ratio = image.shape[1] / float(resized.shape[1])

        # Si no hemos encontrado ninguna coincidencia hasta el momento o la encontrada es mejor que la actual
        # guardamos los resultados
        if found is None or maxVal > found[0]:
            found = (maxVal, maxLoc, ratio, scale)

    # Extraemos los datos de la coincidencia si se encontró alguna
    if found is not None:
        maxVal, maxLoc, ratio, scale = found
        #print("Se ha encontrado una coincidencia en la imagen con un nivel de confianza de:", maxVal)
        # Calculamos las coordenadas del objeto detectado, reescalado para ajustarse a la imagen original
        x_start, y_start = (int(maxLoc[0] * ratio), int(maxLoc[1] * ratio))
        x_end, y_end = (int((maxLoc[0] + template_width) * ratio), int((maxLoc[1] + template_height) * ratio))
        # Usamos np.where para encontrar las posiciones donde se encontró la plantilla
        y_idxs, x_idxs = np.where(result >= 0.95 * np.max(result))

        cv2.normalize(result, result, 0, 100, cv2.NORM_MINMAX) # Normalizamos los valores de la matriz resultante
        _, maxVal, _, maxLoc = cv2.minMaxLoc(result) # Obtenemos el valor máximo de la matriz resultante
        formateado_porcentaje = "{:.2f}".format(maxVal) # Formateamos el porcentaje a dos decimales
    
        #print(formateado_porcentaje)
        # Dibujamos la detección y mostramos el resultado en pantalla
        for y, x in zip(y_idxs, x_idxs):
            cv2.rectangle(image, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
            #cv2.putText(image, f"Confianza: {maxVal}%", (x_start, y_start - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(image, f"Confianza: {maxVal}%", (x_start, y_start - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) # En color verde

        cv2.imshow('Imagen', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        print("No se encontró ninguna coincidencia de plantilla en la imagen.")


## Detección multiple.

> Detecta 9 logos, con un nivel de confianza mayor o igual al 75%

In [1]:
import numpy as np
import cv2
import imutils

# Función para calcular el Índice de Jaccard (IoU) entre dos rectángulos
# La función toma las coordenadas (x, y, w, h) de dos rectángulos y devuelve el IoU
# Referencia: https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
def calculate_iou(boxA, boxB):
    # Obtenemos las coordenadas del rectángulo
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    # Calculamos el área de intersección
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Calculamos el área de cada rectángulo
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    # Calculamos el IoU dividiendo el área de intersección por la unión de áreas
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # Devolvemos el IoU
    return iou

# Cargamos la plantilla y convertirla a escala de grises
template = cv2.imread('../data/template/pattern.png', 0)
template = cv2.Canny(template, 50, 200)
template_height, template_width = template.shape[:2]

# Cargamos la imagen objetivo con múltiples detecciones
image = cv2.imread('../data/images/coca_multi.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Inicializamos una lista para almacenar las detecciones
detections = []

# Iterar sobre diferentes escalas de la imagen
for scale in np.linspace(0.1, 5.0, 400)[::-1]: # Iteramos sobre 200 escalas diferentes
    # Redimensionamos la imagen
    resized = imutils.resize(gray, width=int(gray.shape[1] * scale))

    # Si la imagen redimensionada es más pequeña que la plantilla, detenemos la iteración
    if resized.shape[0] < template_height or resized.shape[1] < template_width:
        break

    # Calculamos el mapa de bordes de la imagen redimensionada
    edged = cv2.Canny(resized, 50, 200)

    # Aplicamos el algoritmo de detección de plantillas
    result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
    _, maxVal, _, maxLoc = cv2.minMaxLoc(result)

    # Calculamos el ratio de redimensionamiento
    ratio = image.shape[1] / float(resized.shape[1])

    # Guardamos la detección
    detections.append((maxVal, maxLoc, ratio))

# Establecemos el umbral de área máxima para eliminar grandes rectángulos
max_area_threshold = 5000

# Filtramos detecciones superpuestas utilizando IoU y el umbral de área máxima
filtered_detections = []

# Obtenemos el máximo valor de confianza de la lista de detecciones
max_confidence = np.max([det[0] for det in detections])  

for detection in detections: # Iteramos sobre las detecciones
    maxVal, maxLoc, ratio = detection
    
    x_start, y_start = int(maxLoc[0] * ratio), int(maxLoc[1] * ratio)
    x_end, y_end = int((maxLoc[0] + template_width) * ratio), int((maxLoc[1] + template_height) * ratio)
    
    area = (x_end - x_start) * (y_end - y_start)

    # Calculamos el porcentaje de confianza utilizando el valor máximo de correlación de plantilla (maxVal)
    confianza = round((maxVal / max_confidence) * 100, 2)

    # Comprobamos si el área supera el umbral máximo y la confianza es alta
    if area <= max_area_threshold and confianza >= 75:
        overlap = False
        for (x, y, w, h, _) in filtered_detections: # Iteramos sobre las detecciones filtradas
            iou = calculate_iou((x_start, y_start, x_end - x_start, y_end - y_start), (x, y, w, h)) # Calculamos el IoU
            if iou > 0.05: 
                overlap = True
                break

        if not overlap: # Si no hay superposición, agregamos la detección filtrada
            filtered_detections.append((x_start, y_start, x_end - x_start, y_end - y_start, confianza))

# Dibujamos las detecciones filtradas en la imagen original
for (x, y, w, h, confianza) in filtered_detections: # Iteramos sobre las detecciones filtradas
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Dibujamos el rectángulo
    cv2.putText(image, f"{confianza}%", (x + 5, y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) # Mostramos el porcentaje de confianza en color verde

# Mostramos la imagen con las detecciones
cv2.imshow('Imagen', image)
cv2.waitKey(0)
cv2.destroyAllWindows()